In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


In [2]:
police_df = pd.read_excel('police_stations.xlsx')

police_df.head()

,Station,Address,Beat
0,Central,"33 Artesian, Houston, TX",2A40
1,Downtown,"1900 Rusk, Houston, TX",1A10
2,North,"9455 West Montgomery, Houston, TX",6B40
3,North Belt,"100 Glenborough, Houston, TX",22B10
4,Northwest,"6000 Teague, Houston, TX",5F30


In [3]:
police_df['City'] = [x.split(',')[1] for x in police_df['Address']]
police_df

,Station,Address,Beat,City
0,Central,"33 Artesian, Houston, TX",2A40,Houston
1,Downtown,"1900 Rusk, Houston, TX",1A10,Houston
2,North,"9455 West Montgomery, Houston, TX",6B40,Houston
3,North Belt,"100 Glenborough, Houston, TX",22B10,Houston
4,Northwest,"6000 Teague, Houston, TX",5F30,Houston
5,Northeast,"8301 Ley Road, Houston, TX",8C30,Houston
6,South Central,"2202 St. Emanuel, Houston, TX",10H50,Houston
7,Eastside,"7525 Sherman, Houston, TX",11H10,Houston
8,Clear Lake,"2855 Bay Area Blvd, Houston, TX",12D50,Houston
9,Southeast,"8300 Mykawa, Houston, TX",14D30,Houston


In [4]:
police_df['State'] = [x.split(',')[2] for x in police_df['Address']]
police_df

,Station,Address,Beat,City,State
0,Central,"33 Artesian, Houston, TX",2A40,Houston,TX
1,Downtown,"1900 Rusk, Houston, TX",1A10,Houston,TX
2,North,"9455 West Montgomery, Houston, TX",6B40,Houston,TX
3,North Belt,"100 Glenborough, Houston, TX",22B10,Houston,TX
4,Northwest,"6000 Teague, Houston, TX",5F30,Houston,TX
5,Northeast,"8301 Ley Road, Houston, TX",8C30,Houston,TX
6,South Central,"2202 St. Emanuel, Houston, TX",10H50,Houston,TX
7,Eastside,"7525 Sherman, Houston, TX",11H10,Houston,TX
8,Clear Lake,"2855 Bay Area Blvd, Houston, TX",12D50,Houston,TX
9,Southeast,"8300 Mykawa, Houston, TX",14D30,Houston,TX


In [5]:
police_df['Address'] = [x.split(',')[0] for x in police_df['Address']]
police_df

,Station,Address,Beat,City,State
0,Central,33 Artesian,2A40,Houston,TX
1,Downtown,1900 Rusk,1A10,Houston,TX
2,North,9455 West Montgomery,6B40,Houston,TX
3,North Belt,100 Glenborough,22B10,Houston,TX
4,Northwest,6000 Teague,5F30,Houston,TX
5,Northeast,8301 Ley Road,8C30,Houston,TX
6,South Central,2202 St. Emanuel,10H50,Houston,TX
7,Eastside,7525 Sherman,11H10,Houston,TX
8,Clear Lake,2855 Bay Area Blvd,12D50,Houston,TX
9,Southeast,8300 Mykawa,14D30,Houston,TX


In [6]:
# Convert Integer to Specific Beat
police_df.loc[police_df.Beat == 170000000000, 'Beat'] = '17E10'

police_df

,Station,Address,Beat,City,State
0,Central,33 Artesian,2A40,Houston,TX
1,Downtown,1900 Rusk,1A10,Houston,TX
2,North,9455 West Montgomery,6B40,Houston,TX
3,North Belt,100 Glenborough,22B10,Houston,TX
4,Northwest,6000 Teague,5F30,Houston,TX
5,Northeast,8301 Ley Road,8C30,Houston,TX
6,South Central,2202 St. Emanuel,10H50,Houston,TX
7,Eastside,7525 Sherman,11H10,Houston,TX
8,Clear Lake,2855 Bay Area Blvd,12D50,Houston,TX
9,Southeast,8300 Mykawa,14D30,Houston,TX


In [7]:
police_df['Address'] = police_df['Address'].replace(' ', '+', regex=True)

police_df

,Station,Address,Beat,City,State
0,Central,33+Artesian,2A40,Houston,TX
1,Downtown,1900+Rusk,1A10,Houston,TX
2,North,9455+West+Montgomery,6B40,Houston,TX
3,North Belt,100+Glenborough,22B10,Houston,TX
4,Northwest,6000+Teague,5F30,Houston,TX
5,Northeast,8301+Ley+Road,8C30,Houston,TX
6,South Central,2202+St.+Emanuel,10H50,Houston,TX
7,Eastside,7525+Sherman,11H10,Houston,TX
8,Clear Lake,2855+Bay+Area+Blvd,12D50,Houston,TX
9,Southeast,8300+Mykawa,14D30,Houston,TX


In [8]:
police_df.loc[police_df.Address == '6000+Teague', 'Address'] = '14421+Sommermeyer+St'
police_df.loc[police_df.Address == '8605+Westplace+Dr', 'Address'] = '11851+S+Gessner+Rd'
police_df.loc[police_df.Address == '6227+Southwest+Freeway', 'Address'] = '6227+Southwest+Fwy'
police_df.loc[police_df.Address == '3915+Rustic+Woods', 'Address'] = '3403+Rivers+Edge'

police_df

,Station,Address,Beat,City,State
0,Central,33+Artesian,2A40,Houston,TX
1,Downtown,1900+Rusk,1A10,Houston,TX
2,North,9455+West+Montgomery,6B40,Houston,TX
3,North Belt,100+Glenborough,22B10,Houston,TX
4,Northwest,14421+Sommermeyer+St,5F30,Houston,TX
5,Northeast,8301+Ley+Road,8C30,Houston,TX
6,South Central,2202+St.+Emanuel,10H50,Houston,TX
7,Eastside,7525+Sherman,11H10,Houston,TX
8,Clear Lake,2855+Bay+Area+Blvd,12D50,Houston,TX
9,Southeast,8300+Mykawa,14D30,Houston,TX


In [11]:
police_lats = []
police_lngs = []
census_tract = []

for street, city in zip(police_df['Address'], police_df['City']):
    try:
        census_url = f'https://geocoding.geo.census.gov/geocoder/geographies/address?street={street}&city={city}&state=TX&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json'
        response = requests.get(census_url)
        census_json = response.json()
    #     print(census_json)
        police_lats.append(census_json['result']['addressMatches'][0]['coordinates']['y'])
        police_lngs.append(census_json['result']['addressMatches'][0]['coordinates']['x'])
        census_tract.append(census_json['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['TRACT'])
        print(f"{street} Latitude: {census_json['result']['addressMatches'][0]['coordinates']['y']}")
        print(f"{street} Longitude: {census_json['result']['addressMatches'][0]['coordinates']['x']}")
        print(f"{street} Census Tract: {census_json['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['TRACT']}")
        print('-------------')                                        
    except:
        print(f'{street}: has failed you')


33+Artesian Latitude: 29.764551
33+Artesian Longitude: -95.36878
33+Artesian Census Tract: 510100
-------------
1900+Rusk Latitude: 29.753761
1900+Rusk Longitude: -95.35599
1900+Rusk Census Tract: 100000
-------------
9455+West+Montgomery Latitude: 29.88037
9455+West+Montgomery Longitude: -95.44648
9455+West+Montgomery Census Tract: 533100
-------------
100+Glenborough Latitude: 29.951307
100+Glenborough Longitude: -95.42141
100+Glenborough Census Tract: 550100
-------------
14421+Sommermeyer+St Latitude: 29.859278
14421+Sommermeyer+St Longitude: -95.54168
14421+Sommermeyer+St Census Tract: 521600
-------------
8301+Ley+Road Latitude: 29.831425
8301+Ley+Road Longitude: -95.27464
8301+Ley+Road Census Tract: 231000
-------------
2202+St.+Emanuel Latitude: 29.742254
2202+St.+Emanuel Longitude: -95.362625
2202+St.+Emanuel Census Tract: 312300
-------------
7525+Sherman Latitude: 29.73425
7525+Sherman Longitude: -95.290825
7525+Sherman Census Tract: 311100
-------------
2855+Bay+Area+Blvd L

In [10]:
police_df['Lat'] = police_lats
police_df['Lng'] = police_lngs
police_df['Census_Tract'] = census_tract

police_df

,Station,Address,Beat,City,State,Lat,Lng,Census_Tract
0,Central,33+Artesian,2A40,Houston,TX,29.764551,-95.368780,510100
1,Downtown,1900+Rusk,1A10,Houston,TX,29.753761,-95.355990,100000
2,North,9455+West+Montgomery,6B40,Houston,TX,29.880370,-95.446480,533100
3,North Belt,100+Glenborough,22B10,Houston,TX,29.951307,-95.421410,550100
4,Northwest,14421+Sommermeyer+St,5F30,Houston,TX,29.859278,-95.541680,521600
5,Northeast,8301+Ley+Road,8C30,Houston,TX,29.831425,-95.274640,231000
6,South Central,2202+St.+Emanuel,10H50,Houston,TX,29.742254,-95.362625,312300
7,Eastside,7525+Sherman,11H10,Houston,TX,29.734250,-95.290825,311100
8,Clear Lake,2855+Bay+Area+Blvd,12D50,Houston,TX,29.580847,-95.105050,340400
9,Southeast,8300+Mykawa,14D30,Houston,TX,29.656624,-95.315420,332400
